In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(30)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/8.kit_part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import re
import html as html_lib

# = = = = = = = = = = = = = = =

from tqdm import tqdm

# = = = = = = = = = = = = = = =

directory_in = './file/7.kit_part_number'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input = df_input[df_input['[Kit] JOIN_MPNTCPC'] != ''].drop_duplicates(['[Kit] JOIN_MPNTCPC'],
                                                                              ignore_index=True)

    df_input['No'] = df_input['No'].astype(int)
    df_input['[Kit] No'] = ['' if no == '' else int(no) for no in df_input['[Kit] No'].tolist()]
    df_input['[Kit] Quantity'] = ['' if quantity == '' else int(quantity) for quantity in df_input['[Kit] Quantity'].tolist()]

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            data = {'dopartsearch': '1',
                    'partsearch[partnum][partsearch_007]': crawler_series['[Kit] Part Number'],
                    'partsearch[manufacturer][partsearch_007]': crawler_series['[Kit] Manufacturer'],
                    'partsearch[parttype][partsearch_007]': crawler_series['[Kit] Type Code'],
                    'partsearch[do][partsearch_007]': '1',
                    'func': 'sendparttabsearch',
                    'payload': '{}',
                    'api_json_request': '1'}

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['[Kit] Url']

                    resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                         data=data,
                                         headers=get_header(),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if 'redirect_to_url' not in dict_ and 'collected_javascript' in dict_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(dict_['searchnoderesults'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_tbody = html.xpath('//tbody[contains(@id, "listingcontainer[__GIP__")]')

            # = = = = = = = = = = = = = = =

            list_dict = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

            # = = = = = = = = = = = = = = =

            list_type_code, list_part_code = [dict_['parttype'] for dict_ in list_dict], [dict_['partkey'] for dict_ in list_dict]

            # = = = = = = = = = = = = = = =

            array_index_type_code = np.where(np.array(list_type_code) == crawler_series['[Kit] Type Code'])[0]

            # = = = = = = = = = = = = = = =

            list_dict_2 = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[__GIP__")]/@value')[0]) for tbody in list_tbody]

            # = = = = = = = = = = = = = = =

            list_part_number, list_manufacturer, list_choose = [dict_['partnumber'] for dict_ in list_dict_2], [dict_['catalogname'] for dict_ in list_dict_2], [dict_['paramdesc'].strip() for dict_ in list_dict_2]

            # = = = = = = = = = = = = = = =

            array_index_part_number, array_index_manufacturer = np.where(np.array(list_part_number) == crawler_series['[Kit] Part Number'])[0], np.where(np.array(list_manufacturer) == crawler_series['[Kit] Manufacturer'])[0]

            # = = = = = = = = = = = = = = =

            array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

            # = = = = = = = = = = = = = = =

            if array_index_part.size == 1:
                index_part = array_index_part[0]
            else:
                raise

            # = = = = = = = = = = = = = = =

            list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]')
            list_note = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
            list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
            list_note = [html_lib.unescape(note) for note in list_note]

            # = = = = = = = = = = = = = = =

            dict_note, i = {}, 0
            for note in list_note:
                if note != '':
                    dict_note[str(i)] = note
                    i += 1

            # = = = = = = = = = = = = = = =

            list_a = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
            info = list_a[0].xpath('./@href')[0].strip() if list_a else ''
            info = info if info.startswith('https://www.rockauto.com/en/moreinfo.php?') else f'https://www.rockauto.com/en/moreinfo.php?pk={list_part_code[index_part]}&pt={list_type_code[index_part]}&Lennon=1'

            # = = = = = = = = = = = = = = =

            list_div = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]/parent::div[@class="listing-text-row"]')
            list_note = [etree.tostring(div).decode('utf-8') for div in list_div]
            list_note = [re.sub(r'<.*?>', '', note) for note in list_note]
            list_note = [re.sub(r' +', ' ', note.strip()) for note in list_note]
            list_note = [html_lib.unescape(note) for note in list_note]

            # = = = = = = = = = = = = = = =

            dict_note_2, i = {}, 0
            for note in list_note:
                if note != '':
                    dict_note_2[str(i)] = note
                    i += 1

            # = = = = = = = = = = = = = = =

            list_list_text = [text.split(',') for td in list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]') for text in td.xpath('./span[contains(@title, "Replaces these Alternate")]/text()')]
            list_text = [text.strip() for list_text in list_list_text for text in list_text if text.strip() != '']
            oem = ';'.join(list(dict.fromkeys(list_text)))

            # = = = = = = = = = = = = = = =

            list_input = list_tbody[index_part].xpath('./tr[1]/td/div[contains(@class, "listing-image-space-occupy")]/input[contains(@id, "jsninlineimg[__GIP__")]')
            list_dict = [json.loads(value) for input_ in list_input for value in input_.xpath('./@value')]

            # = = = = = = = = = = = = = = =

            dict_src, i = {}, 0
            for dict_ in list_dict:
                list_src = dict_.get('Slots', [])
                for src in list_src:
                    dict_src[str(i)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''
                    i += 1

            # = = = = = = = = = = = = = = =

            list_span = list_tbody[index_part].xpath('./tr[1]/td[contains(@id, "listingtd[__GIP__") and contains(@id, "__][price]")]/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][td]")]/span/span[contains(@id, "dprice[__GIP__") and contains(@id, "__][v]")]')
            list_price = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
            list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
            list_price = [html_lib.unescape(price) for price in list_price]
            dict_price = {'0': '; '.join(list_price)}

            # = = = = = = = = = = = = = = =

            list_td = list_tbody[index_part].xpath('./tr[1]/td[contains(@class, "listing-inner-content")]')
            list_price = [etree.tostring(option).decode('utf-8') for td in list_td for option in td.xpath('./div/div[contains(@class, "listing-optionchoice-multiple-container")]/span[contains(@id, "ddrepl[optionchoice[__GIP__") and contains(@id, "__]][container]")]/span[@class="hide-if-js"]/select[contains(@id, "optionchoice[__GIP__") and contains(@id, "__]")]/option[@value!=""]')]
            list_price = [re.sub(r'<.*?>', '', price) for price in list_price]
            list_price = [re.sub(r' +', ' ', price.strip()) for price in list_price]
            list_price = [html_lib.unescape(price) for price in list_price]
            if list_price:
                dict_price['1'] = {}
                for i, price in enumerate(list_price):
                    dict_price['1'][str(i)] = price

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'[Kit] JOIN_MPNTCPC': crawler_series['[Kit] JOIN_MPNTCPC'],
                                          '[Kit] Type': crawler_series['[Kit] Type'],
                                          '[Kit] Manufacturer': crawler_series['[Kit] Manufacturer'],
                                          '[Kit] Part Number': crawler_series['[Kit] Part Number'],
                                          '[Kit] OEM': oem,
                                          '[Kit] Picture': '',
                                          '[Kit] Url': crawler_series['[Kit] Url'],
                                          '[Kit] Json_Src': json.dumps(dict_src),
                                          '[Kit] Info': info,
                                          '[Kit] Json_Note 1': json.dumps(dict_note),
                                          '[Kit] Json_Note 2': json.dumps(dict_note_2),
                                          '[Kit] Choose': list_choose[index_part],
                                          '[Kit] Json_Price': json.dumps(dict_price),
                                          '[Kit] Type Code': crawler_series['[Kit] Type Code'],
                                          '[Kit] Part Code': list_part_code[index_part]})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['[Kit] JOIN_MPNTCPC'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['[Kit] JOIN_MPNTCPC'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-kit_part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：41

[True] - [请求1次] - [剩余40条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 8. https://www.rockauto.com/en/parts/MELLING,BG5623,Timing Chain Guide / Damper,5728

[True] - [请求1次] - [剩余39条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 1. https://www.rockauto.com/en/parts/MELLING,S899,Timing Crank Sprocket,5723

[True] - [请求1次] - [剩余38条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 1. https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724

[True] - [请求1次] - [剩余37条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 8. https://www.rockauto.com/en/parts/MELLING,471,Timing Chain,5724

[True] - [请求1次] - [剩余36条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 6. https://www.rockauto.com/en/parts/MELLING,S1276,Timing Cam Sprocket,5722

[True] - [请求1次] - [剩余35条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 5. https://www.rockauto.com/en/parts/MELLING,BG5054,Timing Chain Guide / Damper,5728

[True] - [请求1次] - [剩余34条] - [crawler_1（100.00%）] - [2024/10/18 10:18:26] - 1. 

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2464.34it/s]

Done ~
